#### This notebook is to *numerically* calculate multitude of random distribution (m.r.d.) expected for $(\phi,\psi)$ diffraction orientations

- Dependents

In [ ]:
pwd

In [ ]:
%pylab inline

import TX, os, shutil
from TX import cmb, euler
euler = euler.euler
path_home=os.getcwd()
# cmb.random

path_load='/Users/yj/repo/rs_pack/src/src_fortran/'
os.chdir(path_load)
import cs
reload(cs)
os.chdir(path_home)

In [ ]:
### Define random orientation
grains=cmb.random(ngrain=1000,phi1=360, phi2=360, phi=360)
print grains.shape

In [ ]:
## rotation matrices for indivial discrete orientations
a_mat=[]
for i in xrange(len(grains)):
    gr=grains[i]
    ph,th,tm = gr[:3]
    a=euler(ph=grains[0][0], th=grains[0][1], tm=grains[0][2],echo=False)
    a_mat.append(a)

In [ ]:
hkls,n=cs.cubic_eqvect(v=[2,1,1])
hkls=hkls.T
hkls=hkls[:n,:]

In [ ]:
print cs.vector_ang.__doc__

In [ ]:
def vecply(a,b):
    c = np.zeros((3,))
    for i in xrange(3):
        for j in xrange(3):
            c[i] = c[i] + b[i][j] * a[j]
    return c

def vnorm(vec):
    v=vec[::]
    mag=0.
    for i in xrange(len(v)):
        mag = mag + v[i]**2
    mag = np.sqrt(mag)
    
    for i in xrange(len(v)):
        v[i] = v[i] / mag
        
    return v

In [ ]:
a=[2,0,2]
v=vnorm(a)
print a
print v

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(221, projection='3d')
ax1 = fig.add_subplot(222)

## phi / psi
phis  = [  0, 45, 90]
betas = [-45,-15, -7,  0,  7, 15, 45]

## angmisdet 
angmisdet = 23.59 ## -- depend on 'hkl' and X-ray tube

x0_R_s      = np.zeros((3,3))
x0_R_s[0,1] = 1.
x0_R_s[1,0] = 1.
x0_R_s[2,2] =-1.

bv0    = np.zeros((3,))
bv0[2] = 1.0  # [0,0,1] 
x      = np.zeros((3,))  # x axis of X-ray coordinate
x[0]   = 1.0

## R p1 in xa <- beam in xa
## R p2 in xa <- beam in xa
p1Rb   = cs.vector_ang(x, +angmisdet/2.)
p2Rb   = cs.vector_ang(x, -angmisdet/2.)
nphi = len(phis)
nbet = len(betas)

for iphi in xrange(nphi):
    for ibeta in xrange(nbet):
        ph = phis[iphi]
        be = betas[ibeta]
        x1_R_x0 = euler(ph=-ph,th=be,tm=0,echo=False)
        bv1     = vecply(bv0,x1_R_x0)        
        x1_R_s  = cs.matply(x1_R_x0, x0_R_s)
        s_R_x1  = x1_R_s.T

        bvs = vecply(bv0, s_R_x1) ## Beam vector in sample axes        
        ps1 = vecply(bv0, p1Rb)
        ps2 = vecply(bv0, p2Rb)        
        ps1 = vecply(ps1, s_R_x1)
        ps2 = vecply(ps2, s_R_x1)

        ax.scatter(ps1[0],ps1[1],ps1[2])
        ax.scatter(ps2[0],ps2[1],ps2[2])
        ax1.plot(ps1[0],ps1[1],'k.')
        ax1.plot(ps2[0],ps2[1],'k.')

ax.set_zlim(0,-1.)